In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


# IMPORT DATA

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
print(df_train.shape)
df_train.head(10)

In [ ]:
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_test.head()

# EDA

In [ ]:
df_train.info()

In [ ]:
pd.DataFrame(df_train.nunique())

In [ ]:
df_train = df_train.drop(columns = ['Name', 'Cabin', 'Ticket'])
df_train.head(15)

In [ ]:
numerical_features = df_train[['Age', 'SibSp', 'Parch', 'Fare']]
categorical_features = df_train[['Pclass', 'Sex', 'Embarked']]

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:

df_train.head(10)

In [ ]:
corr = df_train.drop(columns = 'Survived').select_dtypes('number').corr()
sns.heatmap(corr, annot = True);

In [ ]:
sns.boxplot(x = 'Survived', y='Pclass', data = df_train)
plt.ylabel('passenger Class')
plt.xlabel('Survived')

In [ ]:
sns.boxplot(x = 'Survived', y='Age', data = df_train)
plt.ylabel('Age')
plt.xlabel('Survived')

In [ ]:
df_train['Survived'].value_counts(normalize = True).plot(kind = 'bar')

# BASELINE

In [ ]:
df_train['Survived'].value_counts(normalize = True).max()

In [ ]:
df_train.shape

# MODEL

In [ ]:
y_train = df_train['Survived']
X_train = df_train.drop(columns = 'Survived')
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_test = df_test
print(X_train.shape)
print(y_train.shape)

In [ ]:
model = make_pipeline(
    OneHotEncoder(use_cat_names = True),
    SimpleImputer(),
    RandomForestClassifier(n_estimators = 100, max_depth = 35, random_state = 42)
)


In [ ]:
model.fit(X_train, y_train)

In [ ]:
acc_training = model.score(X_train, y_train)
acc_training

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:

predictions= model.predict(X_test)
predictions

# COMMUNICATION


In [ ]:
features = model.named_steps['onehotencoder'].get_feature_names_out()
importance = model.named_steps['randomforestclassifier'].feature_importances_

feat_imp = pd.Series(importance, index=features).sort_values(key=abs)
feat_imp

In [ ]:
feat_imp.tail(15).plot(kind = 'barh');
plt.xlabel('Importances')
plt.ylabel('Feature')
plt.title('Feature Importances')

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")